In [1]:
import os
import sys
import re
import time
import urllib.request
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}

In [ ]:
# 저장해놓은 naver 주가 데이터에서 날짜 데이터 이용
df = pd.read_csv('stock/naver_stock_data.csv') 
date_list = [i.replace('-', '.') for i in df['Date']] # 형식 변환 (2023-06-01 -> 2023.06.01)

# 기사 제목과 날짜를 저장할 최종 데이터프레임
news_df = pd.DataFrame(columns=['Title', 'Date']) 

# "네이버" URL 인코딩
query = "네이버"
encQuery = urllib.parse.quote(query)

# 날짜마다 관련도 기준 상위 40개 기사 추출 
for date in tqdm(date_list): # 날짜마다 기사 추출 반복 (진행률 표시)
        title_list = [] # 해당 날짜의 40개의 기사 제목을 저장할 리스트
        for start in [1, 11, 21, 31]: # 각 start 마다 10개 기사 추출 (1~10, 11~20 ...)
                date1 = date.replace('.', '') # 형식 변환1 (2023.06.01 -> 20230601)

                # 네이버 뉴스 검색 URL 생성
                url =  "https://search.naver.com/search.naver?where=news"\
                        + f"&query={encQuery}"\
                        + f"&ds={date}&de={date}"\
                        + f"&nso=so%3Ar%2Cp%3Afrom{date1}to{date1}"\
                        + f"&start={start}" # 날짜당 4개씩 url 생성
                
                # 웹 페이지의 HTML 콘텐츠를 가져와 BeautifulSoup으로 파싱
                web = requests.get(url, headers=headers).content
                soup = BeautifulSoup(web, 'html.parser') # BeautifulSoup 객체 soup
                
                # 뉴스 제목 추출 후 title_list에 추가
                ## 'a' 태그 중 클래스가 'news_tit'인 모든 요소를 찾아 텍스트 추출 -> titles 리스트 (url 마다 10개씩)
                titles = [title.text.strip()  for title in soup.find_all('a', attrs={'class':'news_tit'})]
                ## url 마다 10개씩의 제목들을 누적 추가하여 날짜마다 총 40개씩 저장
                title_list.extend(titles)
        
        # 날짜마다 모은 40개의 기사들을 저장할 임시 데이터프레임
        temp_df = pd.DataFrame({
                'Title': title_list, 
                'Date': [date] * len(title_list) # 모은 기사 개수만큼 날짜 리스트 사이즈 맞추기
        })

        # 최종 데이터프레임에 임시 데이터프레임을 넣어서 날짜마다 계속 누적
        news_df = pd.concat([news_df, temp_df], ignore_index=True)

print("----끝----") # 총 9640 (= 241(날짜) * 40(기사))

In [3]:
# 날짜별로 기사가 40개씩 제대로 모아졌나 확인
groupby_news_df = news_df.groupby(['Date'])[['Title']].count()
groupby_news_df['Title'].value_counts() # 성공

Title
40    241
Name: count, dtype: int64

In [4]:
# 기사 제목이 중복되는 행 보기
news_df[news_df.duplicated(subset=['Title'], keep=False)]

# 확인해보니 같은 언론사 -> 무의미하다고 생각하여 삭제

,Title,Date
49,"앳홈, 미닉스 미니 건조기 네이버 쇼핑에서 론칭",2023.06.02
50,"앳홈, 미닉스 미니 건조기 네이버 쇼핑에서 론칭",2023.06.02
169,"“7천원 냈는데, 쓰지도 못하고 환불 받아라?” 네이버 왜 이래?",2023.06.08
171,"“7천원 냈는데, 쓰지도 못하고 환불 받아라?” 네이버 왜 이래?",2023.06.08
217,"고려은단, 네이버 메가세일&쇼핑라이브 진행",2023.06.09
...,...,...
9606,라인 장수게임 잇단 종료…네이버와 거리두나,2024.05.27
9607,"'COO도 지원사격' 네이버 치지직…""팝업스토어 성료, 굿즈 품절 대란도""",2024.05.27
9613,"네이버-카카오, 검색·쇼핑 등 전방위 위기 고조",2024.05.27
9614,라인 장수게임 잇단 종료…네이버와 거리두나,2024.05.27


In [6]:
# 기사 제목 기준 중복되는 행 삭제
news_df = news_df.drop_duplicates(subset=['Title'])

In [8]:
news_df.to_csv('naver_news.csv', index=False)